In [1]:
## Next steps:
## Find data on schedules and results
## Make new table with stats and results
## Make prediction model

In [ ]:

import pandas as pd
import numpy as np

In [2]:
### Data Exploration
np.random.seed(1)
df = pd.read_csv('2020.csv', header=[0,1])
df.head(2)

Unnamed: 0_level_0 Unnamed: 1_level_0 Overall                             \
                  Rk             School       G   W   L   W-L%   SRS   SOS   
0                  1  Abilene Christian      23  13  10  0.565 -2.90 -5.05   
1                  2          Air Force      25   9  16  0.360 -0.53  2.88   

  Conf.      ... School Totals                                                
      W   L  ...            FT  FTA    FT%  ORB  TRB  AST  STL BLK  TOV   PF  
0     8   4  ...           366  496  0.738  262  791  344  212  60  328  485  
1     3  10  ...           387  519  0.746  178  810  354  124  39  305  424  

[2 rows x 34 columns]

In [3]:
### Data Cleaning

### Only want a single header
df = pd.read_csv('2020.csv', header=[1])
df.head(2)

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485
1,2,Air Force,25,9,16,0.360,-0.53,2.88,3,10,...,387,519,0.746,178,810,354,124,39,305,424


In [4]:
#rename w.1 etc
df.rename(columns={"W.1": "Conf W", "W.2": "Home W", "W.3": "Away W"}, inplace=True)
df.rename(columns={"L.1": "Conf L", "L.2": "Home L", "L.3": "Away L"}, inplace=True)
df.head(1)

,Rk,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.9,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485


In [5]:
df.drop(['Unnamed: 16', 'Rk'], axis=1, inplace = True)
df.head(4)

,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,10,...,366,496,0.738,262,791,344,212,60,328,485
1,Air Force,25,9,16,0.360,-0.53,2.88,3,10,6,...,387,519,0.746,178,810,354,124,39,305,424
2,Akron,24,18,6,0.750,7.97,-0.53,8,3,12,...,358,461,0.777,236,896,329,117,68,320,421
3,Alabama A&M,22,6,16,0.273,-18.22,-8.27,3,8,4,...,271,388,0.698,217,757,238,132,44,258,387


In [42]:
### Data Exploration of new table
schedule_df = pd.read_csv('big_ten.csv')
schedule_df[schedule_df['PTS'].isnull()==False].head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes
0,Fri Dec 6 2019,Iowa,91.0,Michigan,103.0,NaN,NaN
1,Sat Dec 7 2019,Illinois,58.0,Maryland,59.0,NaN,NaN
2,Sat Dec 7 2019,Penn State,74.0,Ohio State,106.0,NaN,NaN
3,Sat Dec 7 2019,Indiana,64.0,Wisconsin,84.0,NaN,NaN
4,Sun Dec 8 2019,Rutgers,65.0,Michigan State,77.0,NaN,NaN


In [45]:
schedule_df[schedule_df['Notes'].isnull()==False].head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes


In [51]:
### Drop unneeded columns  My simple model leave out OT for now.
schedule_df.drop(['Date','Notes', 'OT'], axis=1, inplace=True)

In [59]:
schedule_df.tail()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1
135,Rutgers,NaN,Purdue,NaN
136,Iowa,NaN,Illinois,NaN
137,Michigan,NaN,Maryland,NaN
138,Ohio State,NaN,Michigan State,NaN
139,Nebraska,NaN,Minnesota,NaN


In [63]:
#Remove unplayed games
schedule_df = schedule_df[schedule_df['PTS'].isnull()==False]

In [65]:
schedule_df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1
0,Iowa,91.0,Michigan,103.0
1,Illinois,58.0,Maryland,59.0
2,Penn State,74.0,Ohio State,106.0
3,Indiana,64.0,Wisconsin,84.0
4,Rutgers,65.0,Michigan State,77.0


In [70]:

combine = pd.merge(schedule_df,df, left_on='Visitor/Neutral', right_on='School')
combine.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,Home L,Away W,Away L,Tm.,Opp.,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Iowa,91.0,Michigan,103.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372
1,Iowa,86.0,Penn State,89.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372
2,Iowa,70.0,Nebraska,76.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372
3,Iowa,75.0,Northwestern,62.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372
4,Iowa,72.0,Maryland,82.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372


In [72]:
Schedule_and_stats = pd.merge(combine, df, left_on = 'Home/Neutral', right_on = 'School')
Schedule_and_stats.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,Away W_x,Away L_x,Tm._x,Opp._x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,ORB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,School_y,G_y,W_y,L_y,W-L%_y,SRS_y,SOS_y,Conf W_y,Conf L_y,Home W_y,Home L_y,Away W_y,Away L_y,Tm._y,Opp._y,MP_y,FG_y,FGA_y,FG%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,Iowa,91.0,Michigan,103.0,Iowa,24,17,7,0.708,16.55,9.09,8,5,12,1,3,4,1899,1720,960,663,1469,0.451,196,558,0.351,377,505,0.747,289,916,426,157,87,286,372,Michigan,23,14,9,0.609,17.15,10.41,5,7,9,4,1,5,1738,1583,935,642,1398,0.459,195,561,0.348,259,363,0.713,210,845,343,127,109,266,347
1,Illinois,64.0,Michigan,62.0,Illinois,24,16,8,0.667,15.80,8.98,8,5,12,3,4,4,1767,1558,965,636,1419,0.448,134,428,0.313,361,491,0.735,307,966,322,117,67,307,384,Michigan,23,14,9,0.609,17.15,10.41,5,7,9,4,1,5,1738,1583,935,642,1398,0.459,195,561,0.348,259,363,0.713,210,845,343,127,109,266,347
2,Penn State,72.0,Michigan,63.0,Penn State,24,19,5,0.792,18.01,8.10,9,4,12,1,5,3,1861,1623,960,667,1491,0.447,195,582,0.335,332,480,0.692,265,933,355,195,125,280,423,Michigan,23,14,9,0.609,17.15,10.41,5,7,9,4,1,5,1738,1583,935,642,1398,0.459,195,561,0.348,259,363,0.713,210,845,343,127,109,266,347
3,Rutgers,63.0,Michigan,69.0,Rutgers,24,17,7,0.708,15.56,7.95,8,5,16,0,1,5,1694,1474,965,640,1435,0.446,129,437,0.295,285,443,0.643,299,988,316,162,114,305,390,Michigan,23,14,9,0.609,17.15,10.41,5,7,9,4,1,5,1738,1583,935,642,1398,0.459,195,561,0.348,259,363,0.713,210,845,343,127,109,266,347
4,Ohio State,61.0,Michigan,58.0,Ohio State,23,15,8,0.652,18.50,8.46,5,7,10,2,3,5,1647,1416,920,562,1249,0.450,198,522,0.379,325,447,0.727,228,853,321,108,87,305,402,Michigan,23,14,9,0.609,17.15,10.41,5,7,9,4,1,5,1738,1583,935,642,1398,0.459,195,561,0.348,259,363,0.713,210,845,343,127,109,266,347
